In [1]:
import os
import pandas as pd

from dotenv import load_dotenv
from pymongo import MongoClient
from tqdm import tqdm

In [2]:
df_ENGINE = pd.read_excel("sampel data.xls", sheet_name="Engine")

df_SAVONIUS = pd.read_excel("sampel data.xls", sheet_name="Savonius")
df_SAVONIUS = df_SAVONIUS.ffill()
df_SAVONIUS["Bukaan Katup"] = df_SAVONIUS["Bukaan Katup"].str.replace("°", "")
df_SAVONIUS["Bukaan Katup"] = pd.to_numeric(df_SAVONIUS["Bukaan Katup"])

df_CRANK_MECHANISM = pd.read_excel("sampel data.xls", sheet_name="CRANK MECHANISM")
df_CRANK_MECHANISM = df_CRANK_MECHANISM.iloc[1:].reset_index(drop=True)
df_CRANK_MECHANISM.columns = [
    "Sudut Engkol (θ)",
    "Langkah Piston (S)_Praktek (mm)",
    "Langkah Piston (S)_Teori (mm)",
    "Error %",
    "Massa (m) Kg (10^-3)",
    "Torsi_Aktual (Ta)",
    "Torsi_Ideal (Ti)",
    "Kerugian Gesekan %"
]

df_PLTU = pd.read_excel("sampel data.xls", sheet_name="PLTU")

In [3]:
load_dotenv(".env")
client = MongoClient(
    os.getenv("MONGO_CONNECTION_STRING"),
    serverSelectionTimeoutMS=300000
)
db = client[os.getenv("MONGO_DATABASE_NAME")]
collection_ENGINE = db[os.getenv("MONGO_COLLECTION_NAME_ENGINE")]
collection_SAVONIUS = db[os.getenv("MONGO_COLLECTION_NAME_SAVONIUS")]
collection_CRANK_MECHANISM = db[os.getenv("MONGO_COLLECTION_NAME_CRANK_MECHANISM")]
collection_PLTU = db[os.getenv("MONGO_COLLECTION_NAME_PLTU")]

In [4]:
def store_to_mongo(df, collection, mode="append"):
    """
    Stores a dataframe to a MongoDB collection.

    Parameters:
    df (pd.DataFrame): The dataframe to be stored.
    collection (pymongo.collection.Collection): The MongoDB collection.
    mode (str): The mode of operation - 'replace' to replace existing data, 'append' to add to existing data.
    """
    # Convert dataframe to list of dictionaries with progress bar
    data_dict = [row for row in tqdm(df.to_dict(orient="records"), desc="Converting dataframe")]

    if mode == "replace":
        # Delete all existing documents in the collection
        collection.delete_many({})
        print("Existing data replaced.")
    
    # Insert all documents at once
    collection.insert_many(data_dict)
    print(f"{len(data_dict)} documents inserted.")

def load_from_mongo(collection):
    """
    Loads data from a MongoDB collection into a pandas dataframe.

    Parameters:
    collection (pymongo.collection.Collection): The MongoDB collection.

    Returns:
    pd.DataFrame: The loaded dataframe.
    """
    cursor = collection.find()
    df = pd.DataFrame(list(cursor))
    
    # Remove the MongoDB "_id" field if present
    if "_id" in df.columns:
        df.drop("_id", axis=1, inplace=True)
    
    return df

In [5]:
store_to_mongo(df_ENGINE, collection_ENGINE, mode="replace")
store_to_mongo(df_SAVONIUS, collection_SAVONIUS, mode="replace")
store_to_mongo(df_CRANK_MECHANISM, collection_CRANK_MECHANISM, mode="replace")
store_to_mongo(df_PLTU, collection_PLTU, mode="replace")

Converting dataframe: 100%|██████████| 57/57 [00:00<00:00, 749452.44it/s]


Existing data replaced.
57 documents inserted.


Converting dataframe: 100%|██████████| 12/12 [00:00<00:00, 235194.62it/s]


Existing data replaced.
12 documents inserted.


Converting dataframe: 100%|██████████| 19/19 [00:00<00:00, 241490.23it/s]


Existing data replaced.
19 documents inserted.


Converting dataframe: 100%|██████████| 10/10 [00:00<00:00, 103054.15it/s]

Existing data replaced.
10 documents inserted.


In [6]:
df_ENGINE = load_from_mongo(collection_ENGINE)
df_SAVONIUS = load_from_mongo(collection_SAVONIUS)
df_CRANK_MECHANISM = load_from_mongo(collection_CRANK_MECHANISM)
df_PLTU = load_from_mongo(collection_PLTU)